# Datamigos - The Iron Curtain Dirty Movie Secret - Report 2024

- [1 - Dataset and Cleaning](#1---Dataset-and-Cleaning)

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from src.dataset_creation.cmu_merging_cleaning import *
from src.dataset_creation.datasets_merging import *
from src.dataset_creation.gpt_4o_data_enhancement import *
from src.dataset_creation.merged_dataset_preprocessing import *

from src.utils.constants import *
from src.utils.helpers import *

## 1 - Dataset Merging and Cleaning

As described in Milestone 2, our final dataset is composed of the CMU Movie Summary Corpus extanded with two external datasets:

- [The Movies Dataset (TMDb)](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) contains metadata for 45,000 films listed in the Full MovieLens Dataset, all of which were released on or before July 2017. We will use this dataset, specifically the movies_metadata.csv file, primarily to enrich the movie.metadata.tsv of the CMU movie dataset. Additionally, this dataset includes plot keywords (in the keywords.csv file), which will be helpful for analyzing movie plots.

- [IMDB Non-Commercial Dataset](https://developer.imdb.com/non-commercial-datasets/) is a giant database containing over 50 Million movies from different regions, which allows us to complement our initial dataset specifically with movies created in the Soviet Union during the cold war. The CMU Movie Summaries data being largely focused on the United States.

We first load, merge and clean the CMU Movie Summary Corpus dataset:

In [2]:
cmu_dataset = create_plot_summary_dataset()
print(cmu_dataset.shape)
cmu_dataset.sample(5)

(81694, 10)


,wikipedia_id,freebase_id,title,release_date,revenue,runtime,languages,countries,genres,plot_summary
67319,16356812,/m/03whpr7,Ai Margini della Metropoli,1952-01-01,NaN,96.0,[Italian Language],[Italy],"[Crime Fiction, Black-and-white]",NaN
34900,22644734,/m/05zpb_m,Vroeger is dood,1987-01-01,NaN,NaN,[Dutch Language],[Netherlands],[Drama],NaN
60758,30009185,/m/09gqsm2,Den nye lægen,1943-12-26,NaN,91.0,NaN,[Norway],[Drama],NaN
21761,34084031,/m/0hr16v9,The Orphan Killer,2011-11-01,NaN,NaN,NaN,NaN,[Horror],The film follows the life of two children who ...
348,9105523,/m/027xtwf,Picture Bride,1994-05-01,NaN,100.0,[Japanese Language],"[United States of America, Japan]","[Japanese Movies, World cinema, Docudrama, Mar...","The film is set in 1918. Riyo is a ""city girl..."


We now merge TMDb and IMDb datasets with the CMU Movie Summary Corpus dataset and clean the resulting dataset.

In [3]:
merged_dataset = create_merged_dataset()
print(merged_dataset.shape)
merged_dataset.sample(5)

(25621, 10)


,wikipedia_id,freebase_id,title,languages,countries,genres,keywords,release_date,runtime,plot_summary
95072,9036116.0,/m/027vccd,The Cousins,NaN,[Ukraine],[Drama],NaN,1959,108.0,"A pair of cousins share a flat, but animosity ..."
105972,7091498.0,/m/0h3qtp,The Quick and the Dead,[English],"[Japan, United States of America]","[Action, Western]","[gunslinger, revenge, prairie, shootout, pistol]",1975,100.0,A mysterious woman comes to compete in a quick...
54816,6883275.0,/m/0gv4n1,Let's Make Love,"[English, Deutsch, العربية, Italiano, Français]",[United States of America],"[Comedy, Musical, Romance]",[musical],1960,119.0,The plot revolves around billionaire Jean-Marc...
14721,31523947.0,/m/0glspdb,Boardinghouse,NaN,[Soviet Union],"[Comedy, Horror]",NaN,1982,98.0,"On September 18, 1972, the Hoffman house was c..."
89873,NaN,NaN,Taina zelyonogo bora,NaN,[Soviet Union],[Adventure],NaN,1961,NaN,Three boys in a revolutionary Russia village h...


## 2 - Data Enhancement with GPT-4o mini and cleaning

We used the GPT-4o mini model to generate 6 new columns for our dataset. The columns are:
- `cold_war_side` - What side of the cold war the country belongs to if any
- `character_western_bloc_representation` - How the character representing the western block is portrayed 
- `character_eastern_bloc_representation` - How the character representing the eastern block is portrayed
- `eastern_bloc_values` - The values represented by the eastern block
- `western_bloc_values` - The values represented by the western block
- `theme` - The theme and keywords of the movie

This will be very useful later on to grasp the context of the movies and to understand the values and themes they represent.  

this is the prompt we used to generate the new columns:

> You are an expert in movie history and Cold War. You will be given the name of the film, the year and the plot of the movie. You first need to analyse if the movie can be identified to the Eastern or Western bloc during the Cold War. If yes come up with the character or group of character impersonating the Western and Eastern bloc and their values as well as their main archetye.
> Your output needs to be parsable comma separated without context (the output needs to start directly), use only keyword and very important use new line character after each of the following:
> - Cold War side belonging either Easter, Western or None.
> - The character or group of character representing Western bloc with their values and archetype comma separated or None.
> - The character or group of character representing Eastern bloc with their values and archetype comma separated or None.
> - The Western bloc representation main values and characteristics comma separated or None.
> - The Eatern bloc representation main values and characteristics comma separated or None.
> - The theme of the movies and keywords.
> If and only if the movie does not belong to any of the blocs, put `None` in the respective fields.


This is the script we used to call the api and generate the new columns:

In [4]:
import src.prompt_engineering.prompt_engineering
output_4o = pickle.load(open(DATA_FOLDER_PREPROCESSED + 'output4o.pkl', "rb"))
output_4o[0]

'Western  \nJoe Collins, American bank security consultant, resourceful, clever, hero archetype  \nDawn Divine, hooker with a heart of gold, cunning, loyal, anti-heroine archetype  \nNone  \nResourcefulness, cleverness, individualism, heroism  \nNone  \nCrime, heist, betrayal, survival, trust, money, adventure'

We need to parse all the output of GPT and clean it to be able to use it in our dataset.

In [6]:
merged_dataset_enhanced = create_enhanced_dataset()
merged_dataset_enhanced.sample(5)

,wikipedia_id,freebase_id,title,languages,countries,genres,keywords,release_date,runtime,plot_summary,cold_war_side,character_western_bloc_representation,character_eastern_bloc_representation,western_bloc_values,eastern_bloc_values,theme
16825,9810279.0,/m/02psy44,Spring Fever,['English'],['United States of America'],['Comedy'],['golf'],1982,100.0,Two girls competing in a tennis match suddenly...,"""None""",[None],[None],[None],[None],"[competition, friendship, spontaneity, youth, ..."
21395,9287656.0,/m/0283413,The Prisoner of Zenda,"['Español', 'English']",['United States of America'],"['Romance', 'Adventure', 'Drama']","['kidnapping', 'coronation', 'villain', 'kingd...",1952,96.0,An Englishman on a Ruritarian holiday must imp...,"""None""",[None],[None],[None],[None],"[Impersonation, monarchy, adventure, loyalty]"
11762,2001394.0,/m/06d3ll,Midnight Lace,['English'],['United States of America'],"['Thriller', 'Crime', 'Mystery']","['london england', 'suspicion', 'threat to dea...",1960,110.0,Newlywed American heiress Kit Preston is livin...,"""None""",[None],[None],[None],[None],"[Suspense, Isolation, Trust, Betrayal, Mental ..."
20001,NaN,NaN,The Hussars,NaN,['Soviet Union'],"['Comedy', 'War']",NaN,1955,NaN,This satire on militarism is set during the pe...,"""None""",[None],[None],[None],[None],"[Satire, militarism, French Revolution, Napole..."
19368,26286968.0,/m/0b73tbb,The Flaw,['English'],['United Kingdom'],"['Foreign', 'Documentary']","['wall street', 'banking', 'economics', 'finan...",1955,61.0,The story of the credit bubble that caused th...,"""None""",[None],[None],[None],[None],"[Financial crash, credit bubble, interviews, e..."


We now do a final cleaning, processing and drop useless columns of the dataset to prepare it for analysis.

In [8]:
preprocess_movies = create_preprocessed_movies()
preprocess_movies.sample(5)

,title,languages,countries,genres,release_date,cold_war_side,character_western_bloc_representation,character_eastern_bloc_representation,western_bloc_values,eastern_bloc_values,theme
16921,Starlift,"[English, French]",[United States of America],"[Comedy, Drama, Music]",1951,"""Western""","[ friendship, love, duty, archetype: soldiers ...",[None],"[Friendship, love, duty, optimism]",[None],"[Romantic comedy, military, love, friendship, ..."
4221,Cries & Whispers,NaN,[Lithuania],[Drama],1972,"""None""",[None],[None],[None],[None],"[Family, Emotions, Death, Repression, Relation..."
19995,The Hunters,[English],[United States of America],"[Action, Romance, Drama, War]",1958,"""Western""","[Major Cleve ""Iceman"" Saville, Commanding offi...",[None],"[Camaraderie, Bravery, Duty, Sacrifice]",[None],"[Military conflict, Heroism, Sacrifice, Friend..."
18681,The Club,[English],[Russia],"[Comedy, Horror, Drama, Sport]",1994,"""None""",[None],[None],[None],[None],"[Coming of age, self-discovery, morality, iden..."
7026,Game of Seduction,NaN,[Russia],[Drama],1976,"""None""",[None],[None],[None],[None],"[Seduction, Betrayal, Love, Duels, Aristocracy]"


## 3 - Data Exploration